In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def RichPoorClassifer(C):
    if (df.loc["Customer"]==C, Payment (Rs) >200):
      print("You're rich")




df=pd.read_excel("/Lab Session Data.xlsx",usecols=['Customer','Candies (#)','Mangoes (Kg)','Milk Packets (#)','Payment (Rs)'])
print("Dataset loaded:")
print(df)
A=np.array([df['Candies (#)'],df['Mangoes (Kg)'], df['Milk Packets (#)']])
A=np.transpose(A)
print("A:",A)
C=np.array(df['Payment (Rs)'])
C=np.transpose(C)
print("C:",C)
print("Dimensionality of A",A.shape[1])
print("Number of vectors", A.shape[0])
print("Rank of A", np.linalg.matrix_rank(A))
X=np.matmul(np.linalg.pinv(A),C)   #X=A^-1 *B
print("Cost of each product using pseudo inverse:",X)
#A2
print("Model Vector", X)
#A3
RichPoorClassifer(C_1)


Dataset loaded:
  Customer  Candies (#)  Mangoes (Kg)  Milk Packets (#)  Payment (Rs)
0      C_1           20             6                 2           386
1      C_2           16             3                 6           289
2      C_3           27             6                 2           393
3      C_4           19             1                 2           110
4      C_5           24             4                 2           280
5      C_6           22             1                 5           167
6      C_7           15             4                 2           271
7      C_8           18             4                 2           274
8      C_9           21             1                 4           148
9     C_10           16             2                 4           198
A: [[20  6  2]
 [16  3  6]
 [27  6  2]
 [19  1  2]
 [24  4  2]
 [22  1  5]
 [15  4  2]
 [18  4  2]
 [21  1  4]
 [16  2  4]]
C: [386 289 393 110 280 167 271 274 148 198]
Dimensionality of A 3
Number of vectors 10
Ra

NameError: name 'C_1' is not defined